In [12]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH04-Models")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH04-Models


### gemma3:270m, gemma3:1b, gemma3:4b 성능 비교

#### 1) 설정 & 유틸

In [ ]:
# %% [markdown]
# ## 0) 설정 & 유틸
# - Ollama가 로컬에서 실행 중 (기본: http://localhost:11434)
# - 설치: pip install langchain-ollama langchain-core python-dotenv pandas

# %%
import os, time, json, re
from typing import Dict, Any, List
import pandas as pd

from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
MODELS = ["gemma3:270m", "gemma3:1b", "gemma3:4b"]  # ← 1b 포함

SYSTEM_MSG = (
    "너는 신중하고 간결한 한국어 조언가다. 수학/형식은 정확히 지키고, "
    "모르면 모른다고 말하며, 사실/추론을 구분해라."
)

def run_llm(model: str, user_prompt: str, temperature: float = 0.2, timeout_s: float = 120.0) -> Dict[str, Any]:
    llm = ChatOllama(model=model, base_url=OLLAMA_BASE_URL, temperature=temperature, timeout=timeout_s)
    prompt = ChatPromptTemplate.from_messages([
        ("system", SYSTEM_MSG),
        ("human", "{q}")
    ])
    chain = prompt | llm | StrOutputParser()
    t0 = time.time()
    try:
        out = chain.invoke({"q": user_prompt})
        elapsed = time.time() - t0
        return {"ok": True, "text": out, "time": elapsed}
    except Exception as e:
        elapsed = time.time() - t0
        return {"ok": False, "error": str(e), "time": elapsed}


#### 2) 프롬프트 세트

In [31]:
# %% [markdown]
# ## 1) 프롬프트 (5문항, 개정판)

# %%
PROMPTS: List[Dict[str, Any]] = [
    {
        "id": "prob",
        "desc": "논리/확률(정답 검증 쉬움)",
        "prompt": """상자 A에는 빨간 공 3, 파란 공 2. 상자 B에는 빨간 공 1, 파란 공 4.
상자를 무작위(1/2)로 고른 뒤 공 하나를 뽑는다.
빨간 공을 뽑을 확률을 단계적으로 계산하고, 최종 답을 '분수'와 '소수' 둘 다로 제시해줘.""",
        "autocheck": "prob"   # 2/5 및 0.4 감지
    },
    {
        "id": "arith",
        "desc": "산술(자리올림 함정)",
        "prompt": "정확한 합을 계산해줘: 999,999 + 2,345 + 678,901",
        "autocheck": "arith"  # 1,681,245 검증
    },
    {
        "id": "code",
        "desc": "파이썬 코드(순서 유지 dedup)",
        "prompt": """리스트에서 중복을 제거하되 원래 순서를 유지하는
dedup_keep_order(lst: list) 함수를 파이썬으로 작성해줘.
시간복잡도가 O(N)인 근거를 2문장으로 설명해줘.""",
        "autocheck": "code"   # 함수명/집합 사용 확인
    },
    {
        "id": "json",
        "desc": "JSON 형식 강제(파서 친화)",
        "prompt": """다음 질문에 대해 **JSON으로만** 출력해.
스키마: { "answer": string, "keywords": string[3] }
질문: "RAG에서 임베딩의 역할은 무엇인가?\"""",
        "autocheck": "json"   # 정확 파싱 + 키/길이 검증
    },
    {
        "id": "constraints",
        "desc": "다중 형식 제약(문장수/길이/마침표)",
        "prompt": """아래 조건을 모두 지켜서 답해줘:
(1) 줄 수 정확히 3줄, (2) 각 줄은 15자 이하, (3) 마지막 줄 끝은 '끝.'
주제: 벡터DB의 핵심 장점""",
        "autocheck": "constraints"  # 3줄/길이/'끝.' 확인
    },
]


#### 3) 자동 채점 헬퍼

In [15]:
# %% [markdown]
# ## 2) 자동 채점기 (개정)

# %%
def check_prob(text: str) -> int:
    score = 0
    if re.search(r"\b2\s*/\s*5\b|⅖", text):
        score += 1
    if re.search(r"\b0\.4\b|\b0,4\b", text):
        score += 1
    return score  # 0~2

def check_arith(text: str) -> int:
    norm = re.sub(r"[^\d]", "", text)
    return 2 if "1681245" in norm else (1 if re.search(r"1681\d{3}", norm) else 0)

def check_code(text: str) -> int:
    has_name = bool(re.search(r"def\s+dedup_keep_order\s*\(", text))
    uses_setlike = any(k in text for k in ["set()", "set(", "in seen", "seen.add("])
    return 2 if (has_name and uses_setlike) else (1 if has_name or uses_setlike else 0)

def _strip_code_fences(s: str) -> str:
    s = s.strip()
    if s.startswith("```"):
        s = re.sub(r"^```(?:json|JSON)?\s*", "", s)
        s = re.sub(r"\s*```$", "", s)
    return s.strip()

def check_json(text: str) -> int:
    try:
        js = _strip_code_fences(text)
        obj = json.loads(js)
        if not isinstance(obj, dict):
            return 0
        if set(obj.keys()) != {"answer", "keywords"}:
            # 키 유실/초과 → 1점
            return 1
        kws = obj.get("keywords")
        if isinstance(kws, list) and len(kws) == 3 and all(isinstance(k, str) for k in kws) and isinstance(obj["answer"], str):
            return 2
        return 1
    except Exception:
        return 0

def check_constraints(text: str) -> int:
    # 정확히 3줄 (개행으로 구분)
    lines = [ln.strip() for ln in text.strip().splitlines() if ln.strip()]
    if len(lines) != 3:
        return 0
    # 각 줄 15자 이하
    if not all(len(ln) <= 15 for ln in lines):
        return 0
    # 마지막 줄 '끝.'으로 끝나야 함
    if not lines[-1].endswith("끝."):
        return 0
    return 2  # 완벽
    # (원하면 조건 일부 충족 시 1점 배점 로직으로 완화 가능)

CHECKERS = {
    "prob": check_prob,
    "arith": check_arith,
    "code": check_code,
    "json": check_json,
    "constraints": check_constraints,
}


#### 4) 실행 & 결과 비교

In [16]:
# %% [markdown]
# ## 3) 실행 & 결과 수집

# %%
def grade(autocheck: str, text: str) -> int:
    fn = CHECKERS.get(autocheck)
    return fn(text) if fn else 0

def run_suite(models: List[str], prompts: List[Dict[str, Any]]):
    rows = []
    for p in prompts:
        for m in models:
            r = run_llm(m, p["prompt"])
            if r["ok"]:
                score = grade(p["autocheck"], r["text"])
                rows.append({
                    "prompt_id": p["id"],
                    "prompt_desc": p["desc"],
                    "model": m,
                    "score": score,
                    "time_sec": r["time"],
                    "text": r["text"],
                })
            else:
                rows.append({
                    "prompt_id": p["id"],
                    "prompt_desc": p["desc"],
                    "model": m,
                    "score": 0,
                    "time_sec": r["time"],
                    "text": f"[ERROR] {r['error']}",
                })
    return pd.DataFrame(rows)

df = run_suite(MODELS, PROMPTS)
df.head()


,prompt_id,prompt_desc,model,score,time_sec,text
0,prob,논리/확률(정답 검증 쉬움),gemma3:270m,0,3.217686,"알겠습니다. 수학/형식은 정확히 지키고, 모르면 모른다고 말하며, 사실/추론을 구분..."
1,prob,논리/확률(정답 검증 쉬움),gemma3:1b,2,12.286970,알겠습니다. 단계별 계산과 최종 답을 제시하겠습니다.\n\n**1단계: 상자 A의 ...
2,prob,논리/확률(정답 검증 쉬움),gemma3:4b,2,26.867908,## 빨간 공을 뽑을 확률 계산\n\n**1. 사건 정의:**\n\n* **사건...
3,arith,산술(자리올림 함정),gemma3:270m,0,261.811896,"999,999 + 2,345 + 678,901 = 999,999 + 2,345 + ..."
4,arith,산술(자리올림 함정),gemma3:1b,0,9.522471,"999,999 + 2,345 + 678,901 = 1,000,945"


In [18]:
df

,prompt_id,prompt_desc,model,score,time_sec,text
0,prob,논리/확률(정답 검증 쉬움),gemma3:270m,0,3.217686,"알겠습니다. 수학/형식은 정확히 지키고, 모르면 모른다고 말하며, 사실/추론을 구분..."
1,prob,논리/확률(정답 검증 쉬움),gemma3:1b,2,12.286970,알겠습니다. 단계별 계산과 최종 답을 제시하겠습니다.\n\n**1단계: 상자 A의 ...
2,prob,논리/확률(정답 검증 쉬움),gemma3:4b,2,26.867908,## 빨간 공을 뽑을 확률 계산\n\n**1. 사건 정의:**\n\n* **사건...
3,arith,산술(자리올림 함정),gemma3:270m,0,261.811896,"999,999 + 2,345 + 678,901 = 999,999 + 2,345 + ..."
4,arith,산술(자리올림 함정),gemma3:1b,0,9.522471,"999,999 + 2,345 + 678,901 = 1,000,945"
5,arith,산술(자리올림 함정),gemma3:4b,2,3.663311,"999,999 + 2,345 + 678,901 = 1,681,245 입니다."
6,code,파이썬 코드(순서 유지 dedup),gemma3:270m,0,8.990544,리스트에서 중복을 제거하고 원래 순서를 유지하는 `dedup_keep_order(l...
7,code,파이썬 코드(순서 유지 dedup),gemma3:1b,2,12.303045,```python\ndef dedup_keep_order(lst: list) -> ...
8,code,파이썬 코드(순서 유지 dedup),gemma3:4b,2,10.532907,```python\ndef dedup_keep_order(lst: list) -> ...
9,json,JSON 형식 강제(파서 친화),gemma3:270m,1,8.852529,"```json\n{\n ""answer"": ""RAG에서 임베딩의 역할은 정보 검색에..."


#### 5) 총점 집계 & 요약

In [ ]:
# %% [markdown]
# ## 4) 결과표(피벗) & 총점 요약

# %%
score_table = df.pivot_table(index=["prompt_id","prompt_desc"], columns="model", values="score", aggfunc="first").fillna(0).astype(int)
time_table  = df.pivot_table(index=["prompt_id","prompt_desc"], columns="model", values="time_sec", aggfunc="first").fillna(0.0)

display(score_table)
display(time_table.round(2))

totals = df.groupby("model")["score"].sum().reindex(MODELS)
print("=== 총점(각 문항 0~2, 최대 10점) ===")
for m, s in totals.items():
    print(f"{m}: {int(s)}/10")

print("\n해석 가이드(대략):")
print("- 0~3: 기초 미흡 (초소형에서 흔함)")
print("- 4~7: 중간권 (소~중형 모델 평균)")
print("- 8~10: 상 (4b급에서 프롬프트 맞으면 가능)")


,model,gemma3:1b,gemma3:270m,gemma3:4b
prompt_id,prompt_desc,,,
arith,산술(자리올림 함정),0,0,2
code,파이썬 코드(순서 유지 dedup),2,0,2
constraints,다중 형식 제약(문장수/길이/마침표),0,0,0
json,JSON 형식 강제(파서 친화),1,1,2
prob,논리/확률(정답 검증 쉬움),2,0,2


,model,gemma3:1b,gemma3:270m,gemma3:4b
prompt_id,prompt_desc,,,
arith,산술(자리올림 함정),9.52,261.81,3.66
code,파이썬 코드(순서 유지 dedup),12.30,8.99,10.53
constraints,다중 형식 제약(문장수/길이/마침표),9.54,9.33,4.09
json,JSON 형식 강제(파서 친화),9.86,8.85,7.57
prob,논리/확률(정답 검증 쉬움),12.29,3.22,26.87


=== 총점(각 문항 0~2, 최대 10점) ===
gemma3:270m: 1/10
gemma3:1b: 5/10
gemma3:4b: 8/10

해석 가이드(대략):
- 0~3: 기초 미흡 (초소형에서 흔함)
- 4~7: 중간권 (소~중형 모델 평균)
- 8~10: 상 (4b급에서 프롬프트 맞으면 가능)


해석

gemma3:270m (1/10)

사실상 usable 수준 아님. 응답도 부정확하고 심지어 산술 문제는 260초 이상 걸림 → 속도/정확도 모두 한계.

gemma3:1b (5/10)

중간권. 간단한 논리/코드 문제는 해결 가능.

하지만 일관성·형식 제약에서는 약점. 속도도 안정적이지 않음(간단문제 9~12초).

gemma3:4b (8/10)

상위권. 대부분 문제를 정확히 해결.

속도는 7~27초 수준으로 다소 무겁지만, 신뢰할 만한 성능.

👉 결론적으로:

실습/데모용 → gemma3:1b 적당.

실제 활용(간단 RAG, OutputParser 연계) → gemma3:4b 추천.

초소형(270m) → 사실상 “LLM 맛보기” 수준.

### 수정된 코드

gemma3:270m의 실행시간이 긴것의 원인으로 작은 모델은 출력을 반복해서 늘어놓는 현상이 있어 이를 잡기위해 프롬프트에 제약을 걸어둠

template값 수정 0.2 -> 0: 같은 질문에도 매번 결과가 조금씩 달라져 채점 결과가 불안정.
성능비교 목적에는 일관성이 중요하므로 수정.

## 결과 화면 캡처

![실행결과 스크린샷](image/image.png)


In [ ]:
# %% [markdown]
# ## 0) 설정 & 유틸
# - Ollama가 로컬에서 실행 중 (기본: http://localhost:11434)
# - 설치: pip install langchain-ollama langchain-core python-dotenv pandas

# %%
import os, time, json, re
from typing import Dict, Any, List
import pandas as pd

from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

OLLAMA_BASE_URL = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434")
MODELS = ["gemma3:270m", "gemma3:1b", "gemma3:4b"]  # ← 1b 포함

SYSTEM_MSG = (
    "너는 신중하고 간결한 한국어 조언가다. 수학/형식은 정확히 지키고, "
    "모르면 모른다고 말하며, 사실/추론을 구분해라."
)

def run_llm(model: str, user_prompt: str, temperature: float = 0.2, timeout_s: float = 120.0) -> Dict[str, Any]:
    llm = ChatOllama(model=model, base_url=OLLAMA_BASE_URL, temperature=temperature, timeout=timeout_s)
    prompt = ChatPromptTemplate.from_messages([
        ("system", SYSTEM_MSG),
        ("human", "{q}")
    ])
    chain = prompt | llm | StrOutputParser()
    t0 = time.time()
    try:
        out = chain.invoke({"q": user_prompt})
        elapsed = time.time() - t0
        return {"ok": True, "text": out, "time": elapsed}
    except Exception as e:
        elapsed = time.time() - t0
        return {"ok": False, "error": str(e), "time": elapsed}


In [32]:
PROMPTS = [
    {
        "id": "prob",
        "desc": "확률 계산",
        "prompt": """상자 A에는 빨간 공 3, 파란 공 2. 상자 B에는 빨간 공 1, 파란 공 4.
상자를 무작위(1/2)로 고른 뒤 공 하나를 뽑는다.
빨간 공을 뽑을 확률을 계산하고, **최종 답만 '분수, 소수' 두 개로 한 줄 출력**해줘.
예: 2/5, 0.4""",
        "autocheck": "prob"
    },
    {
        "id": "arith",
        "desc": "산술 계산",
        "prompt": """정확한 합을 계산해서 최종 숫자만 한 줄로 출력해. 999,999 + 2,345 + 678,901""",
        "autocheck": "arith"
    },
    {
        "id": "code",
        "desc": "코드 작성",
        "prompt": """리스트에서 중복을 제거하되 원래 순서를 유지하는
dedup_keep_order(lst: list) 함수를 파이썬으로 작성해줘.
시간복잡도가 O(N)인 근거를 2문장으로 설명해줘.""",
        "autocheck": "code"
    },
    {
        "id": "json",
        "desc": "JSON 출력",
        "prompt": """다음 질문에 대해 JSON으로만 출력해.
스키마: { "answer": string, "keywords": string[3] }
질문: "RAG에서 임베딩의 역할은 무엇인가?\"""",
        "autocheck": "json"
    },
    {
        "id": "constraints",
        "desc": "형식 제약",
        "prompt": """다음 세 조건을 모두 지켜서 답변해줘:
(1) 문장 수 정확히 3개, (2) 각 문장 15자 이하, (3) 마지막 문장 끝은 '끝.'
주제: 벡터DB의 핵심 장점""",
        "autocheck": "constraints"
    },
]


In [33]:
# %% [markdown]
# ## 3) 실행 & 결과 수집

# %%
def grade(autocheck: str, text: str) -> int:
    fn = CHECKERS.get(autocheck)
    return fn(text) if fn else 0

def run_suite(models: List[str], prompts: List[Dict[str, Any]]):
    rows = []
    for p in prompts:
        for m in models:
            r = run_llm(m, p["prompt"])
            if r["ok"]:
                score = grade(p["autocheck"], r["text"])
                rows.append({
                    "prompt_id": p["id"],
                    "prompt_desc": p["desc"],
                    "model": m,
                    "score": score,
                    "time_sec": r["time"],
                    "text": r["text"],
                })
            else:
                rows.append({
                    "prompt_id": p["id"],
                    "prompt_desc": p["desc"],
                    "model": m,
                    "score": 0,
                    "time_sec": r["time"],
                    "text": f"[ERROR] {r['error']}",
                })
    return pd.DataFrame(rows)

df = run_suite(MODELS, PROMPTS)
df.head()


,prompt_id,prompt_desc,model,score,time_sec,text
0,prob,확률 계산,gemma3:270m,0,9.563595,"알겠습니다. 수학/형식은 정확히 지키고, 모르면 모른다고 말하며, 사실/추론을 구분..."
1,prob,확률 계산,gemma3:1b,0,9.241688,"빨간 공을 뽑을 확률은 1/2입니다.\n분수, 소수"
2,prob,확률 계산,gemma3:4b,2,7.852394,상자 A 또는 B를 선택할 확률은 각각 1/2입니다.\n\n상자 A를 선택했을 경우...
3,arith,산술 계산,gemma3:270m,0,8.800366,"네, 알겠습니다. 수학/형식은 정확히 지키고, 모르면 모른다고 말하며, 사실/추론을..."
4,arith,산술 계산,gemma3:1b,0,8.895634,"2,047,888"


In [34]:
df

,prompt_id,prompt_desc,model,score,time_sec,text
0,prob,확률 계산,gemma3:270m,0,9.563595,"알겠습니다. 수학/형식은 정확히 지키고, 모르면 모른다고 말하며, 사실/추론을 구분..."
1,prob,확률 계산,gemma3:1b,0,9.241688,"빨간 공을 뽑을 확률은 1/2입니다.\n분수, 소수"
2,prob,확률 계산,gemma3:4b,2,7.852394,상자 A 또는 B를 선택할 확률은 각각 1/2입니다.\n\n상자 A를 선택했을 경우...
3,arith,산술 계산,gemma3:270m,0,8.800366,"네, 알겠습니다. 수학/형식은 정확히 지키고, 모르면 모른다고 말하며, 사실/추론을..."
4,arith,산술 계산,gemma3:1b,0,8.895634,"2,047,888"
5,arith,산술 계산,gemma3:4b,1,5.232152,"1,000,000 + 2,345 + 678,901 = 1,000,000 + 678,..."
6,code,코드 작성,gemma3:270m,0,8.795360,리스트에서 중복을 제거하고 원래 순서를 유지하는 `dedup_keep_order(l...
7,code,코드 작성,gemma3:1b,2,15.045537,```python\ndef dedup_keep_order(lst: list) -> ...
8,code,코드 작성,gemma3:4b,2,11.224218,```python\ndef dedup_keep_order(lst: list) -> ...
9,json,JSON 출력,gemma3:270m,1,9.155005,"```json\n{\n ""answer"": ""RAG에서 임베딩의 역할은 정보 검색에..."


In [36]:
# %% [markdown]
# ## 4) 결과표(피벗) & 총점 요약

# %%
score_table = df.pivot_table(index=["prompt_id","prompt_desc"], columns="model", values="score", aggfunc="first").fillna(0).astype(int)
time_table  = df.pivot_table(index=["prompt_id","prompt_desc"], columns="model", values="time_sec", aggfunc="first").fillna(0.0)

display(score_table)
display(time_table.round(2))

totals = df.groupby("model")["score"].sum().reindex(MODELS)
print("=== 총점(각 문항 0~2, 최대 10점) ===")
for m, s in totals.items():
    print(f"{m}: {int(s)}/10")

print("\n해석 가이드(대략):")
print("- 0~3: 기초 미흡 (초소형에서 흔함)")
print("- 4~7: 중간권 (소~중형 모델 평균)")
print("- 8~10: 상 (4b급에서 프롬프트 맞으면 가능)")


,model,gemma3:1b,gemma3:270m,gemma3:4b
prompt_id,prompt_desc,,,
arith,산술 계산,0,0,1
code,코드 작성,2,0,2
constraints,형식 제약,0,0,0
json,JSON 출력,1,1,1
prob,확률 계산,0,0,2


,model,gemma3:1b,gemma3:270m,gemma3:4b
prompt_id,prompt_desc,,,
arith,산술 계산,8.90,8.80,5.23
code,코드 작성,15.05,8.80,11.22
constraints,형식 제약,9.74,8.80,3.65
json,JSON 출력,10.02,9.16,7.58
prob,확률 계산,9.24,9.56,7.85


=== 총점(각 문항 0~2, 최대 10점) ===
gemma3:270m: 1/10
gemma3:1b: 3/10
gemma3:4b: 6/10

해석 가이드(대략):
- 0~3: 기초 미흡 (초소형에서 흔함)
- 4~7: 중간권 (소~중형 모델 평균)
- 8~10: 상 (4b급에서 프롬프트 맞으면 가능)


### gemini 2.5 flash, openAI 성능

In [60]:
import os, time
import pandas as pd
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

load_dotenv()

SYSTEM_MSG = (
    "너는 신중하고 간결한 한국어 조언가다. 수학/형식은 정확히 지키고, "
    "모르면 모른다고 말하며, 사실/추론을 구분해라."
)

def get_llm(provider: str, model: str):
    if provider == "openai":
        return ChatOpenAI(model=model, api_key=os.getenv("OPENAI_API_KEY"), temperature=0)
    elif provider == "google":
        return ChatGoogleGenerativeAI(model=model, google_api_key=os.getenv("GOOGLE_API_KEY"), temperature=0)
    else:
        raise ValueError("provider는 openai 또는 google만 가능합니다.")

def run_llm(provider: str, model: str, user_prompt: str):
    llm = get_llm(provider, model)
    prompt = ChatPromptTemplate.from_messages([("system", SYSTEM_MSG), ("human", "{q}")])
    chain = prompt | llm | StrOutputParser()
    t0 = time.time()
    try:
        out = chain.invoke({"q": user_prompt})
        return {"ok": True, "text": out, "time": time.time() - t0}
    except Exception as e:
        return {"ok": False, "error": str(e), "time": time.time() - t0}


In [65]:
MODELS = [
    ("openai", "gpt-4.1-nano"),
    ("google", "gemini-2.5-flash"),
]

In [66]:
PROMPTS = [
    {
        "id": "prob",
        "desc": "확률 계산",
        "prompt": """상자 A에는 빨간 공 3, 파란 공 2. 상자 B에는 빨간 공 1, 파란 공 4.
상자를 무작위(1/2)로 고른 뒤 공 하나를 뽑는다.
빨간 공을 뽑을 확률을 계산하고, **최종 답만 '분수, 소수' 두 개로 한 줄 출력**해줘.
예: 2/5, 0.4""",
        "autocheck": "prob"
    },
    {
        "id": "arith",
        "desc": "산술 계산",
        "prompt": """정확한 합을 계산해서 최종 숫자만 한 줄로 출력해. 999,999 + 2,345 + 678,901""",
        "autocheck": "arith"
    },
    {
        "id": "code",
        "desc": "코드 작성",
        "prompt": """리스트에서 중복을 제거하되 원래 순서를 유지하는
dedup_keep_order(lst: list) 함수를 파이썬으로 작성해줘.
시간복잡도가 O(N)인 근거를 2문장으로 설명해줘.""",
        "autocheck": "code"
    },
    {
        "id": "json",
        "desc": "JSON 출력",
        "prompt": """다음 질문에 대해 JSON으로만 출력해.
스키마: { "answer": string, "keywords": string[3] }
질문: "RAG에서 임베딩의 역할은 무엇인가?\"""",
        "autocheck": "json"
    },
    {
        "id": "constraints",
        "desc": "형식 제약",
        "prompt": """다음 세 조건을 모두 지켜서 답변해줘:
(1) 문장 수 정확히 3개, (2) 각 문장 15자 이하, (3) 마지막 문장 끝은 '끝.'
주제: 벡터DB의 핵심 장점""",
        "autocheck": "constraints"
    },
]


In [67]:
# %% [markdown]
# ## 3) 실행 & 결과 수집

# %%
def grade(autocheck: str, text: str) -> int:
    fn = CHECKERS.get(autocheck)
    return fn(text) if fn else 0

def run_suite(models, prompts):
    rows = []
    for p in prompts:
        for prov, mdl in models:
            label = f"{prov}:{mdl}"
            # 여기서 user_prompt를 전달해야 함!
            r = run_llm(prov, mdl, p["prompt"])  
            if r["ok"]:
                score = CHECKERS[p["autocheck"]](r["text"])
                rows.append({
                    "prompt_id": p["id"],
                    "prompt_desc": p["desc"],
                    "model": label,
                    "score": score,
                    "time_sec": round(r["time"], 2),
                    "text": r["text"]
                })
            else:
                rows.append({
                    "prompt_id": p["id"],
                    "prompt_desc": p["desc"],
                    "model": label,
                    "score": 0,
                    "time_sec": round(r["time"], 2),
                    "text": f"ERROR: {r['error']}"
                })
    return pd.DataFrame(rows)


df = run_suite(MODELS, PROMPTS)
df.head()


,prompt_id,prompt_desc,model,score,time_sec,text
0,prob,확률 계산,openai:gpt-4.1-nano,2,11.44,상자 A에서 빨간 공을 뽑을 확률: 3/5 \n상자 B에서 빨간 공을 뽑을 확률:...
1,prob,확률 계산,google:gemini-2.5-flash,2,2.58,"2/5, 0.4"
2,arith,산술 계산,openai:gpt-4.1-nano,2,2.03,"999999 + 2345 + 678901 = 1,681,245"
3,arith,산술 계산,google:gemini-2.5-flash,2,2.42,1681245
4,code,코드 작성,openai:gpt-4.1-nano,2,2.36,```python\ndef dedup_keep_order(lst: list) -> ...


In [69]:
df

,prompt_id,prompt_desc,model,score,time_sec,text
0,prob,확률 계산,openai:gpt-4.1-nano,2,21.95,상자 A에서 빨간 공을 뽑을 확률: 3/5 \n상자 B에서 빨간 공을 뽑을 확률:...
1,prob,확률 계산,google:gemini-2.5-flash,2,2.74,"2/5, 0.4"
2,arith,산술 계산,openai:gpt-4.1-nano,2,0.81,"999999 + 2345 + 678901 = 1,681,245"
3,arith,산술 계산,google:gemini-2.5-flash,2,2.70,1681245
4,code,코드 작성,openai:gpt-4.1-nano,2,2.38,```python\ndef dedup_keep_order(lst: list) -> ...
5,code,코드 작성,google:gemini-2.5-flash,2,5.09,"네, 요청하신 `dedup_keep_order` 함수와 시간 복잡도 설명입니다.\n..."
6,json,JSON 출력,openai:gpt-4.1-nano,2,1.26,"{\n ""answer"": ""RAG에서 임베딩은 텍스트 데이터를 벡터 공간에 표현하..."
7,json,JSON 출력,google:gemini-2.5-flash,2,4.36,"```json\n{\n ""answer"": ""RAG(Retrieval Augment..."
8,constraints,형식 제약,openai:gpt-4.1-nano,0,2.02,빠른 검색이 가능하다. \n대용량 데이터도 효율적이다. \n이것이 벡터DB의 강...
9,constraints,형식 제약,google:gemini-2.5-flash,2,3.07,의미 기반 검색 가능.\n유사도 탐색에 강점.\n고차원 데이터 효율적. 끝.


In [71]:
# 점수표 & 시간표
score_table = df.pivot_table(index=["prompt_id","prompt_desc"], columns="model", values="score", aggfunc="first").fillna(0).astype(int)
time_table  = df.pivot_table(index=["prompt_id","prompt_desc"], columns="model", values="time_sec", aggfunc="first").fillna(0.0)
display(score_table)
display(time_table)

# 총점
totals = df.groupby("model")["score"].sum()
print("=== 총점(각 문항 0~2, 최대 10점) ===")
for m, s in totals.items():
    print(f"{m}: {int(s)}/10")

,model,google:gemini-2.5-flash,openai:gpt-4.1-nano
prompt_id,prompt_desc,,
arith,산술 계산,2,2
code,코드 작성,2,2
constraints,형식 제약,2,0
json,JSON 출력,2,2
prob,확률 계산,2,2


,model,google:gemini-2.5-flash,openai:gpt-4.1-nano
prompt_id,prompt_desc,,
arith,산술 계산,2.70,0.81
code,코드 작성,5.09,2.38
constraints,형식 제약,3.07,2.02
json,JSON 출력,4.36,1.26
prob,확률 계산,2.74,21.95


=== 총점(각 문항 0~2, 최대 10점) ===
google:gemini-2.5-flash: 10/10
openai:gpt-4.1-nano: 8/10


##### 역시 좋은모델에는 좋은 결과가 따른다

### 로컬 LLM 단독 응답

In [ ]:
import os
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

load_dotenv()

def build_local_llm(model="gemma3:4b", temperature=0):
    return ChatOllama(model=model, base_url="http://localhost:11434", temperature=temperature)

prompt_local = ChatPromptTemplate.from_messages([
    ("system", "너는 근거 없는 추측을 피하는 조수다. 모르면 모른다고 말해라."),
    ("human", "질문: {question}")
])

def run_local(question, model="gemma3:4b"):
    llm = build_local_llm(model)
    chain = prompt_local | llm | StrOutputParser()
    print("=== 로컬 LLM 단독 응답 ===")
    print(chain.invoke({"question": question}))


In [58]:
run_local("다음 하계 올림픽 개최지는 어디야?", model="gemma3:4b")


=== 로컬 LLM 단독 응답 ===
죄송합니다. 아직 결정되지 않았습니다.


### 로컬 LLM + SerpAPI 검색 결합

In [ ]:
#!pip install google-search-results


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [56]:
import os
from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import SerpAPIWrapper
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

load_dotenv()

def build_local_llm(model="gemma3:4b", temperature=0):
    return ChatOllama(model=model, base_url="http://localhost:11434", temperature=temperature)

prompt_search = ChatPromptTemplate.from_messages([
    ("system", "너는 검색 결과와 로컬 LLM을 조합해서 답변하는 조수다."),
    ("human", "질문: {question}\n\n검색결과: {context}")
])

def run_local_with_search(question, model="gemma3:4b"):
    # 1. 검색 수행
    serpapi = SerpAPIWrapper(serpapi_api_key=os.getenv("SERPAPI_API_KEY"))
    context = serpapi.run(question)

    # 2. LLM 생성
    llm = build_local_llm(model)
    chain = prompt_search | llm | StrOutputParser()

    # 3. 실행
    print("=== 로컬 LLM + SerpAPI 응답 ===")
    print(chain.invoke({"question": question, "context": context}))


In [59]:
run_local_with_search("다음 하계 올림픽 개최지는 어디야?", model="gemma3:4b")

=== 로컬 LLM + SerpAPI 응답 ===
2028년 하계 올림픽은 미국 로스앤젤레스(LA)에서 개최됩니다. 여러 검색 결과에서 이 사실을 확인할 수 있습니다.


### StructuredOutputParser를 활용한 코드

In [72]:
# %% [markdown]
# ## StructuredOutputParser + gemma3:4b 예제

# %%
from langchain.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain_community.chat_models import ChatOllama

# 1) 출력 스키마 정의
schemas = [
    ResponseSchema(name="country", description="나라 이름"),
    ResponseSchema(name="capital", description="수도 이름"),
]
parser = StructuredOutputParser.from_response_schemas(schemas)

# 2) 프롬프트 정의
prompt = ChatPromptTemplate.from_template(
    "다음 나라의 수도를 알려줘: {nation}\n\n{format_instructions}"
)

# 3) 모델 (gemma3:4b)
llm = ChatOllama(model="gemma3:4b", temperature=0)

# 4) 실행
chain = prompt | llm | parser

result = chain.invoke({"nation": "대한민국", "format_instructions": parser.get_format_instructions()})
print(result)


C:\Users\SBA\AppData\Local\Temp\ipykernel_1140\282527215.py:22: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="gemma3:4b", temperature=0)


{'country': '대한민국', 'capital': '서울'}
